# Bert-based Adversarial Examples

## 0.Settings

In [ ]:
!pip install sentence_transformers

In [66]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer
import time

###  0-1. data load & preprocessing

In [57]:
test_data_path = "./data/wellness.user_chatbot.16k.test.q"
sentences = []
with open(test_data_path, 'rt', encoding="UTF8") as f:
    for line in f.readlines():
        sentences.append(line.strip())
sentences[:5]

['저는 어떡하면 좋죠?',
 '내가 집에서는 전혀 존중 받지 못한다는 느낌을 받았어.',
 '너무 불안해서 양손을 꽉 잡았어.',
 '무서워서 이를 악 물었는데 얼마나 세게 물었는지 나중에는 턱이 얼얼하더라구.',
 '다 자기 일이 아니라서 모르는 거예요.']

## 0-2. load model
- [xlm-r-100langs-bert-base-nli-stsb-mean-tokens](https://huggingface.co/sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens)

In [13]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

## 1. Calculate Token Importance
- $Token\  Importance : I_i$
- $Original\  ith\ sentence :  S_i$
- $I_i = 1 - cos\_similarity(S_i, S_i - \{t_n\}$)

In [60]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [65]:
s2 = sentences[2]
tokens = s2.split()
tokens

['너무', '불안해서', '양손을', '꽉', '잡았어.']

In [48]:
start = time.time()
orig_q_emb = model.encode(test_q)
comps = []
for i in range(len(tokens)):
    comp_token = tokens[0:i] + tokens[i+1:]
    comp_q = (' ').join(comp_token)
    comp_q_emb = model.encode(comp_q)
    token_importance = 1 - cos_sim(orig_q_emb, comp_q_emb)
    
    print(test_q, comp_q, cos_similarity , sep ='\t')
print(time.time()-start)

너무 불안해서 양손을 꽉 잡았어.	불안해서 양손을 꽉 잡았어.	0.97398496
너무 불안해서 양손을 꽉 잡았어.	너무 양손을 꽉 잡았어.	0.6588547
너무 불안해서 양손을 꽉 잡았어.	너무 불안해서 꽉 잡았어.	0.86368245
너무 불안해서 양손을 꽉 잡았어.	너무 불안해서 양손을 잡았어.	0.9801966
너무 불안해서 양손을 꽉 잡았어.	너무 불안해서 양손을 꽉	0.4583666
44.94779109954834
